# Tarefa 5 - Treinando Modelos Densos Estáticos
## Alunos
- Yago César
- Samuel Morais


## Imports e logging

In [8]:
import ast
from time import time
import logging
import pandas as pd
from gensim.models.word2vec import Word2Vec
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

## Lendo o corpus pré-processado

In [9]:
df = pd.read_csv("data/TokenizedSentences.csv")

# Transformando as strings "[...strings]" para listas de strings
df['text'] = df['text'].apply(ast.literal_eval)

corpus = df['text'].tolist()
df['text']

0        [nada, queira, voltar, tempo, refazer, vida, t...
1        [lidando, mudanças, humor, tido, muitas, mudan...
2        [noiva, usar, voce, ja, sabe, pergunte, nao, f...
3        [muitas, vezes, consigo, trabalhar, cerca, hor...
4        [quais, ações, sentindo, ansioso, procurando, ...
                               ...                        
50962    [vivo, outras, fazendo, péssimo, trabalho, viv...
50963    [parabéns, amigo, tapa, buraco, depende, quer,...
50964    [ansiedade, doente, epiléptico, ansiedade, def...
50965    [passei, período, difícil, durante, últimos, 3...
50966    [sair, terapia, desde, 14, anos, finalmente, d...
Name: text, Length: 50967, dtype: object

## Modelo Word2Vec
### Treinamento

#### Opções

In [ ]:
# 'skip-gram' ou 'cbow'
TRAINING_ALGORITHM = 'skip-gram'

In [ ]:


w2v_model = Word2Vec(
    vector_size=300, # dimensionalidade do vetor de palavras
    window=5,        # distância máxima entre a palavra atual e a palavra predita em uma sentença
    min_count=5,     # ignora todas as palavras com uma frequência total menor que esse valor
    sample=1e-5,     # limiar para configurar quais palavras de maior frequência são aleatoriamente downsampled
    sg=1 if TRAINING_ALGORITHM == 'skip-gram' else 0 # CBOW = 0 ; skip-gram = 1
)

t = time()
w2v_model.build_vocab(corpus, progress_per=10000)
print(f'Time to build vocab: {round((time() - t) / 60, 2)}')

t = time()
# report_delay - número de segundos para esperar antes de reportar o progresso
w2v_model.train(corpus, total_examples=w2v_model.corpus_count, epochs=30, report_delay=1)
print(f'Time to train the model: {round((time() - t) / 60, 2)} mins')

w2v_model.save('word2vec_{TRAINING_ALGORITHM}.model')


Time to build vocab: 0.01
Time to train the model: 2.21 mins


/tmp/ipykernel_28877/2822979235.py:21: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


### Exploração semântica
#### Palavras mais similares

Tomando como base o contexto do corpus, as 4 palavras escohidas foram:
- depressão;
- ansiedade;
- medo;
- suicídio.

In [ ]:
w2v_model.wv.most_similar(positive=["depressão"], topn=5)

[('ataques', 0.8192235231399536),
 ('ataque', 0.7734318971633911),
 ('ansiedade', 0.6586445569992065),
 ('pânicoansiedade', 0.5356523990631104),
 ('ansioso', 0.4834514856338501),
 ('sintomas', 0.45120304822921753),
 ('tendo', 0.45071545243263245),
 ('batimentos', 0.4454892873764038),
 ('ansiedadepânico', 0.44178271293640137),
 ('cardíaco', 0.4200853109359741)]

In [ ]:
w2v_model.wv.most_similar(positive=["ansiedade"], topn=5)

[('depressão', 0.6619756817817688),
 ('pânico', 0.6586444973945618),
 ('social', 0.657738983631134),
 ('ataques', 0.608400821685791),
 ('diagnosticado', 0.5388233661651611)]

In [ ]:
w2v_model.wv.most_similar(positive=["medo"], topn=5)

[('posso', 0.5136370658874512),
 ('causa', 0.5071543455123901),
 ('morrer', 0.4897680878639221),
 ('preocupado', 0.4877718985080719),
 ('ideia', 0.48593705892562866)]

In [ ]:
w2v_model.wv.most_similar(positive=["suicídio"], topn=5)

[('matar', 0.6484552621841431),
 ('suicida', 0.5514351725578308),
 ('cometer', 0.5420529842376709),
 ('tentativa', 0.5321749448776245),
 ('morte', 0.5142024755477905)]

#### Similaridade semântica entre palavras

Palavras escolhidas:
- ansiedade;
- depressão;
- suicídio.

In [ ]:
w2v_model.wv.similarity('ansiedade', 'depressão')

0.6619757

In [ ]:
w2v_model.wv.similarity('suicídio', 'ansiedade')

0.14742094

In [ ]:
w2v_model.wv.similarity('depressão', 'suicídio')

0.36058834

#### Analogia semântica

Analogias escolhidas:
- **"suicídio + depressão"**: entender quais palavras entram no contexto da combinação entre **"suicídio"** e **"depressão"**.
- **"depressão + ansiedade - suicídio"**: entender o que há em comum entre **depressão** e **ansiedade** sem levar em conta desfechos extremos (**suicídio**).
- **"psicólogo + saúde"**: buscar palavras que representem o papel do **psicólogo** na melhora da **saúde** dos pacientes.

In [ ]:
# suicídio + depressão
w2v_model.wv.most_similar(positive=["suicídio", "depressão"])

[('suicida', 0.6331512331962585),
 ('matar', 0.5774985551834106),
 ('suicidas', 0.5762591361999512),
 ('deprimido', 0.5468285083770752),
 ('mental', 0.5226360559463501),
 ('diagnosticado', 0.5204373598098755),
 ('vida', 0.5046091675758362),
 ('ansiedade', 0.49066269397735596),
 ('transtorno', 0.47831130027770996),
 ('pensamentos', 0.4756019711494446)]

In [ ]:
# depressão + ansiedade - suicídio
w2v_model.wv.most_similar(positive=["depressão", "ansiedade"], negative=['suicídio'])

[('social', 0.5230732560157776),
 ('pânico', 0.4886881709098816),
 ('ataques', 0.4742621183395386),
 ('diagnosticado', 0.4650210738182068),
 ('transtorno', 0.454802006483078),
 ('estresse', 0.44036513566970825),
 ('recentemente', 0.43963363766670227),
 ('severa', 0.4338925778865814),
 ('sintomas', 0.4259622097015381),
 ('geral', 0.42242783308029175)]

In [ ]:
# psicólogo + saúde
w2v_model.wv.most_similar(positive=["psicólogo", "saúde"])

[('mental', 0.6679843664169312),
 ('terapeuta', 0.6156370639801025),
 ('médico', 0.5743861198425293),
 ('psiquiatra', 0.5581804513931274),
 ('terapia', 0.5561575889587402),
 ('consultar', 0.5146474838256836),
 ('ajuda', 0.5139002203941345),
 ('problemas', 0.4710984528064728),
 ('encaminhou', 0.4589638411998749),
 ('ajudar', 0.4547247588634491)]

## Modelo Doc2Vec
### Treinamento

In [24]:
# Para treinar o modelo, será necessário associar uma tag/número com cada documento do corpus
documents = [TaggedDocument(words=tokens, tags=[str(i)]) for i, tokens in enumerate(corpus)]

d2v_model = Doc2Vec(
    vector_size=300,
    window=5,
    min_count=5,
    epochs=30
)

t = time()
d2v_model.build_vocab(documents, progress_per=10000)
print(f'Time to build vocab: {round((time() - t) / 60, 2)}')

t = time()
d2v_model.train(documents, total_examples=d2v_model.corpus_count, epochs=d2v_model.epochs, report_delay=1)
print(f'Time to train the model: {round((time() - t) / 60, 2)} mins')

d2v_model.save('models/doc2vec_model.model')

2025-10-19 23:26:48,647 : INFO : Doc2Vec lifecycle event {'params': 'Doc2Vec<dm/m,d300,n5,w5,mc5,s0.001,t3>', 'datetime': '2025-10-19T23:26:48.647123', 'gensim': '4.4.0', 'python': '3.11.2 (main, Apr 28 2025, 14:11:48) [GCC 12.2.0]', 'platform': 'Linux-6.1.0-40-amd64-x86_64-with-glibc2.36', 'event': 'created'}
2025-10-19 23:26:48,651 : INFO : collecting all words and their counts
2025-10-19 23:26:48,652 : INFO : PROGRESS: at example #0, processed 0 words (0 words/s), 0 word types, 0 tags
2025-10-19 23:26:48,739 : INFO : PROGRESS: at example #10000, processed 609329 words (7043991 words/s), 39103 word types, 10000 tags
2025-10-19 23:26:48,807 : INFO : PROGRESS: at example #20000, processed 1203381 words (8855569 words/s), 56838 word types, 20000 tags
2025-10-19 23:26:48,881 : INFO : PROGRESS: at example #30000, processed 1810721 words (8274186 words/s), 71456 word types, 30000 tags
2025-10-19 23:26:48,955 : INFO : PROGRESS: at example #40000, processed 2414286 words (8151457 words/s), 8

Time to build vocab: 0.01


2025-10-19 23:26:50,517 : INFO : EPOCH 0 - PROGRESS: at 28.43% examples, 820700 words/s, in_qsize 5, out_qsize 0
2025-10-19 23:26:51,518 : INFO : EPOCH 0 - PROGRESS: at 57.20% examples, 828894 words/s, in_qsize 5, out_qsize 0
2025-10-19 23:26:52,528 : INFO : EPOCH 0 - PROGRESS: at 86.39% examples, 835633 words/s, in_qsize 5, out_qsize 0
2025-10-19 23:26:52,939 : INFO : EPOCH 0: training on 3081926 raw words (2925712 effective words) took 3.4s, 852090 effective words/s
2025-10-19 23:26:53,946 : INFO : EPOCH 1 - PROGRESS: at 29.65% examples, 864331 words/s, in_qsize 5, out_qsize 0
2025-10-19 23:26:54,953 : INFO : EPOCH 1 - PROGRESS: at 59.40% examples, 861949 words/s, in_qsize 6, out_qsize 0
2025-10-19 23:26:55,954 : INFO : EPOCH 1 - PROGRESS: at 90.72% examples, 879010 words/s, in_qsize 6, out_qsize 0
2025-10-19 23:26:56,255 : INFO : EPOCH 1: training on 3081926 raw words (2926142 effective words) took 3.3s, 883010 effective words/s
2025-10-19 23:26:57,266 : INFO : EPOCH 2 - PROGRESS: a

Time to train the model: 1.74 mins


### Exploração semântica
Montando queries para comparar com os documentos do corpus.

In [15]:
query = ['sentir', 'ansioso', 'trabalho']
# Infere-se um vetor de similaridade para a query utilizando o modelo.
query_vector = d2v_model.infer_vector(query)

In [23]:
similar_docs = d2v_model.dv.most_similar([query_vector], topn=5)
for tag, similarity in similar_docs:
    print(f"Documento {tag} -> Similaridade: {similarity} <<{' '.join(documents[int(tag)].words)}>>")

Documento 40274 -> Similaridade: 0.7841165065765381 <<alguma dica ansiedade trabalho poderia usar alguns conselhos>>
Documento 26034 -> Similaridade: 0.7557452321052551 <<acompanho trabalho desde início 5potssempre fãabraço>>
Documento 16062 -> Similaridade: 0.7436557412147522 <<ótima vida filhos esposa trabalho carreira resta arrependimento>>
Documento 50087 -> Similaridade: 0.7339484691619873 <<desmoronei trabalho recuperar disso>>
Documento 40861 -> Similaridade: 0.729849636554718 <<ansioso sábado>>
